In [141]:
import pandas as pd
import numpy as np
import os

In [121]:
test = pd.read_csv(r'D:\Test\trips\2022-09-24-01-30-00.csv')
test = test.iloc[:5]

In [122]:
filelist = os.listdir(r'D:\Test')
filelist_ = []
for f in filelist:
    if len(f.split('.')) > 1:
        if f.split('.')[1] == 'pcap':
            filelist_.append(f)

In [144]:
filelist_

['2022-09-23-17-30-00.pcap', '2022-10-02-19-00-00.pcap']

In [123]:
date = [f.split('.')[0] for f in filelist_]
date_ = []
for d in date:
    if d[-1] == 'R':
        date_.append(d[:-2])
    else:
        date_.append(d)
date_ = pd.to_datetime(pd.Series(date_),format=('%Y-%m-%d-%H-%M-%S'))

In [124]:
Ts_records = pd.to_datetime(test.Timestamp,format=('%Y-%m-%d %H:%M:%S'))

In [166]:
Pcap_inds = []
for i in range(len(Ts_records)):
    
    TimeDiff = (Ts_records.iloc[i] - date_)
    within30 = ((Ts_records.iloc[i] - date_) < pd.Timedelta(30,unit='Minute')) & (((Ts_records.iloc[i] - date_) > pd.Timedelta(0,unit='Minute')))
    if within30.sum() == 0:
        Pcap_ind = -1
    else:
        Pcap_ind = TimeDiff.loc[within30].argmin()
    Pcap_inds.append(Pcap_ind)
Pcap_inds = np.array(Pcap_inds)
uni_ind = np.unique(Pcap_inds)
target_frames = []
for i in uni_ind:
    if i == -1:
        continue
    start_frames = np.array(test.loc[Pcap_inds==i].FrameIndex - 30).reshape(-1,1)
    end_frames = np.array(test.loc[Pcap_inds==i].FrameIndex + 30).reshape(-1,1)
    start_frames[start_frames < 0] = 0
    end_frames[end_frames > 17999] = 17999
    target_frames.append(np.concatenate([start_frames,end_frames],axis = 1))